# Convolutional Neural Network

### Importing the libraries

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.__version__

'2.12.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [3]:
train_datagen = ImageDataGenerator(
    rescale = 1.0/255,
    shear_range = 0.2, # this make the image tilt/slant in different direction
    zoom_range = 0.2,
    horizontal_flip = True
)

training_set = train_datagen.flow_from_directory(
    'dataset/training_set',
    target_size = (150,150), #width and height of image that will be fed to convolutional neural network,
    batch_size = 32,
    class_mode = 'categorical' # since output is just 2 classes class_mode is binary, if it is anymore than 2 then categorical should be used.
)

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [4]:
test_datagen = ImageDataGenerator(
    rescale = 1./255,
)

test_set = test_datagen.flow_from_directory(
    'dataset/test_set',
    target_size = (150,150),
    batch_size = 32,
    class_mode = 'categorical'
)

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [5]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [6]:
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, activation = 'relu', input_shape = [150,150,3]))
# here filters are the number of feature detectors that will be taken 
# kernel size is the size of feature detector(size 3 as in 3X3 matrix)
# input shape inlcludes 3 things [target size, target size, (3 or 1) 3 for colored images and 1 for black and white ]


### Step 2 - Pooling

In [7]:
# applying max pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size= 2, strides = 2))

### Adding a second convolutional layer

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, activation = 'relu'))
# input shape for convolution layer is added only for the first layer and not for any other layers
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2))

### Step 3 - Flattening

In [9]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [10]:
cnn.add(tf.keras.layers.Dense(units = 128, activation = 'relu'))
# relu -> rectifier activation function
# here this is creating a hidden layer and there is 
# no specific unit count for this layer, although as it is computer vision and
# invovles pixels, it is recommended to choose a higher unit count

### Step 5 - Output Layer

In [11]:
cnn.add(tf.keras.layers.Dense(units = 2, activation = 'softmax'))
# this is an output layer so, the activation function is sigmoid
# if it is a multiclass classfification, activation would be "softmax"

## Part 3 - Training the CNN

### Compiling the CNN

In [12]:
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])


### Training the CNN on the Training set and evaluating it on the Test set

In [13]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 40)
# over here we are both training the model using training data and 
# evaluating the model using test data

Epoch 1/40
250/250 [==============================] - 88s 347ms/step - loss: 0.6926 - accuracy: 0.5947 - val_loss: 0.6618 - val_accuracy: 0.6255
Epoch 2/40
250/250 [==============================] - 86s 344ms/step - loss: 0.6301 - accuracy: 0.6506 - val_loss: 0.6201 - val_accuracy: 0.6835
Epoch 3/40
250/250 [==============================] - 90s 362ms/step - loss: 0.5962 - accuracy: 0.6850 - val_loss: 0.5839 - val_accuracy: 0.6975
Epoch 4/40
250/250 [==============================] - 89s 357ms/step - loss: 0.5724 - accuracy: 0.7013 - val_loss: 0.5887 - val_accuracy: 0.7010
Epoch 5/40
250/250 [==============================] - 88s 351ms/step - loss: 0.5447 - accuracy: 0.7219 - val_loss: 0.5545 - val_accuracy: 0.7280
Epoch 6/40
250/250 [==============================] - 91s 362ms/step - loss: 0.5144 - accuracy: 0.7489 - val_loss: 0.5431 - val_accuracy: 0.7325
Epoch 7/40
250/250 [==============================] - 93s 373ms/step - loss: 0.4996 - accuracy: 0.7536 - val_loss: 0.5292 - val_ac

## saving our model

In [60]:
tf.keras.models.save_model(cnn,'./saved_model/')

INFO:tensorflow:Assets written to: ./saved_model/assets


INFO:tensorflow:Assets written to: ./saved_model/assets


## Part 4 - Making a single prediction

In [86]:
training_set.class_indices 
# this gives us a result which shows what result (or what number (0,1)) 
# corresponds to which class (dog,cat)

{'cats': 0, 'dogs': 1}

In [96]:
import numpy as np
from keras.utils import load_img, img_to_array
test_image = load_img('dataset/single_prediction/cat_or_dog_2.jpg',
                            target_size = (150,150))
test_image = img_to_array(test_image) # converting PIL image instance to array
test_image = np.expand_dims(test_image, axis = 0)
# expand dims is used because the model was trained in batches and it expects 
# the test-img to also be in a batch, so are manipulating the tensor by adding
# an extra dimension, an extra dimension is being added as the CNN model can
# recognize the extra dimension as a batch
model = tf.keras.models.load_model('./saved_model/')
result = model.predict(test_image/255.0)
# 255.0 as the image needs to be normalized which means essentially
# pixel range must be b/w 0 & 1

# since we added a dimension, the result cannot be directly accessed.
# instead we first get into the dimension which is the first bracket [0]
# and then our result lies in the first value (we are dealing with single image 
# so there will only be one value inside the second bracket)
result = tf.argmax(result,axis=-1)



1/1 [==============================] - 0s 51ms/step


In [98]:
print(result.numpy())

[0]


In [99]:
for animal,number in training_set.class_indices.items():
    if number == result.numpy()[0]:
        print(animal)
    

cats
